In [1]:
%load_ext autoreload
%autoreload 2

# Standard imports
from pathlib import Path
import sys

# Library imports
import matplotlib.pyplot as plt
import xarray as xr

# Util imports
sys.path.append("../../")
import src.climate_downscaling_utils as cd

# Correct station data

This notebook applies bias correction algorithms on gridded data using station data.

**Prerequisite**: Run `notebooks/02-apply-bias-correction/01_overlay_station.ipynb`

### Set input parameters

In [20]:
CITY_NAME = "Dagupan"
DATE = "2008-07-01"
SHOULD_DEBUG = False

PROCESSED_PATH = Path("../../data/02-processed")
CORRECTED_PATH = PROCESSED_PATH / "bias-correction"

STATION_NC = CORRECTED_PATH / f"station_{CITY_NAME.lower()}.nc"
GRIDDED_NC = (
    PROCESSED_PATH
    / f"input/chirts_chirps_regridded_interpolated_{CITY_NAME.lower()}.nc"
)
GRIDDED_SUBSET_NC = CORRECTED_PATH / f"gridded_{CITY_NAME.lower()}.nc"

### Set run parameters

In [22]:
variable_params = dict(
    tmin="CHIRTS minimum temperature",
    # tmax="CHIRTS maximum temperature",
    # precip="CHIRPS precipitation",
)

algo_params = [
    dict(
        name="Liu et al. (2019)",
        func=cd.correct_gridded_liu,
    ),
    # dict(
    #    name="Z-Score",
    #    func=cd.correct_gridded_zscore,
    # ),
]

### Load data

In [5]:
station_ds = xr.open_dataset(STATION_NC, engine="scipy")
gridded_ds = xr.open_dataset(GRIDDED_NC, engine="scipy")
gridded_subset_ds = xr.open_dataset(GRIDDED_SUBSET_NC, engine="scipy")

In [8]:
station_lat = station_ds["lat"].item()
station_lon = station_ds["lon"].item()

# Apply bias correction

In [25]:
for var, title in variable_params.items():
    print(f"Now doing {title}")

    gridded_da = gridded_ds[var].sel(time=DATE, method="nearest")
    gridded_subset_da = gridded_subset_ds[var].sel(time=DATE, method="nearest")

    if SHOULD_DEBUG:
        gridded_da.plot()
        plt.plot(station_lon, station_lat, "o")
        plt.show()

        gridded_subset_da.plot()
        plt.plot(station_lon, station_lat, "o")
        plt.show()

        gridded_subset_da.plot.hist(bins=15)
        plt.show()

    for algo_param in algo_params:
        print(f"Now doing {algo_param['name']} bias correction")

        corrected_da = algo_param["func"](
            gridded_subset_da,
            station_da=station_ds[var].sel(time=DATE, method="nearest"),
            std_scale=0.1,
            should_plot=SHOULD_DEBUG,
        )

        if SHOULD_DEBUG:
            plot_min = min([corrected_da.min(), gridded_subset_da.min()]).values
            plot_max = max([corrected_da.max(), gridded_subset_da.max()]).values

            gridded_subset_da.plot(vmin=plot_min, vmax=plot_max)
            plt.title(title)
            plt.show()

            corrected_da.plot(vmin=plot_min, vmax=plot_max)
            plt.title(f"Corrected {title}\n{algo_param['name']}")
            plt.show()

            (corrected_da - gridded_subset_da).plot(cmap="RdBu")
            plt.title(
                f"Difference between corrected and uncorrected\n{title}\n{algo_param['name']}"
            )
            plt.show()

Now doing CHIRTS minimum temperature
Now doing Liu et al. (2019) bias correction


In [26]:
corrected_da

<xarray.DataArray 'tmin' (lat: 13, lon: 13)>
array([[31.40536523, 31.31750062, 31.23492184, 31.08198955, 30.93163401,
        30.80774333, 30.72997382, 30.78526711, 30.98505816, 31.32440751,
        31.76589461, 32.24369511, 32.7796155 ],
       [31.75814593, 31.65041064, 31.54742305, 31.36421566, 31.17239276,
        30.98678774, 30.7947481 , 30.7294915 , 30.80363206, 31.02538206,
        31.37603611, 31.82580367, 32.35935016],
       [32.16052627, 32.0423986 , 31.92796886, 31.72989247, 31.51453877,
        31.2817613 , 30.9779848 , 30.78978972, 30.73001834, 30.81674563,
        31.05527492, 31.47939699, 32.03798235],
       [31.45593911, 31.49381337, 31.53246649, 31.5718836 , 31.61204992,
        31.65295078, 31.25432094, 30.95717782, 30.77549514, 30.73254882,
        30.8467753 , 31.20065191, 31.75118614],
       [        nan,         nan,         nan, 31.96023277, 31.95764187,
        31.95504919, 31.48495616, 31.1123043 , 30.85376042, 30.73544665,
        30.77705097, 31.05679186, 31.55439144],
       [        nan,         nan,         nan,         nan,         nan,
                nan, 31.74874721, 31.30266081, 30.96832674, 30.77328193,
        30.73809051, 30.93720962, 31.37483052],
       [        nan,         nan,         nan,         nan,         nan,
                nan,         nan, 31.17105974, 31.06050756, 30.81025242,
        30.73052287, 30.89976773, 31.33428077],
       [        nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan, 31.12779642, 30.83387507,
        30.72965664, 30.90324951, 31.37476357],
       [        nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan, 30.73919276,
        30.72960019, 30.91987578, 31.43851664],
       [        nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
        30.74971375, 31.06179903, 31.69812766],
       [        nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
        30.80415668, 31.23877657, 31.98799472],
       [        nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
        30.89938276, 31.49945913, 32.42332308],
       [        nan,         nan,         nan,         nan,         nan,
                nan,         nan,         nan,         nan,         nan,
        31.03716369, 31.84054877, 32.98184749]])
Coordinates:
    time         datetime64[ns] 2008-07-01
  * lon          (lon) float64 120.2 120.2 120.3 120.3 ... 120.4 120.4 120.5
  * lat          (lat) float64 15.97 15.99 16.01 16.03 ... 16.17 16.19 16.21
    band         int32 1
    spatial_ref  int32 0